## Imports

In [ ]:
from __future__ import annotations

import pprint
import re
import struct
from array import array
from dataclasses import dataclass, field
from enum import Enum
from itertools import chain
from pathlib import Path
from typing import BinaryIO
from warnings import warn

# Please install PIL or Pillow:
from PIL.ImagePalette import ImagePalette
from PIL.Image import frombytes as Image_frombytes, Image, Transpose

## Shared / library code

In [ ]:
class HoverRaceTrackException(Exception):
    pass

In [ ]:
# Why isn't this a built-in function already?

def unpack_stream(format: str, stream: BinaryIO):
    size = struct.calcsize(format)
    buf = stream.read(size)
    assert len(buf) == size
    assert isinstance(buf, bytes)
    return struct.unpack(format, buf)    

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Parcel/ClassicObjStream.cpp

MAX_STRING_LEN = 16 * 1024

def pack_string_length(size: int) -> bytes:
    if size < 0xff:
        return struct.pack('<B', size)
    elif size < 0xfffe:
        return struct.pack('<BH', 0xff, size)
    else:
        return struct.pack('<BHL', 0xff, 0xffff, size)

def unpack_string_length(stream: BinaryIO) -> int:
    (b,) = unpack_stream('<B', stream)
    if b < 0xff:
        return b
    (w,) = unpack_stream('<H', stream)
    if w == 0xfffe:
        # 0xfffe is the marker for Unicode strings.
        raise NotImplementedError('ClassicObjStream::ReadStringLength for unicode strings')
    elif w == 0xffff:
        (dw,) = unpack_stream('<L', stream)
        return dw
    else:
        return w

def pack_string(s: str) -> bytes:
    b = s.encode('ascii')
    size = len(b)
    if size > MAX_STRING_LEN:
        # Note: this logic will lead to broken last character if we use utf-8.
        warn('String length {} exceeds max {}; truncated: {}...'.format(size, MAX_STRING_LEN, s[:64]))
        size = MAX_STRING_LEN
    return pack_string_length(size) + b[:size]

def unpack_string(stream: BinaryIO) -> str:
    size = unpack_string_length(stream)
    excess = 0
    if size > MAX_STRING_LEN:
        warn('String length {} exceeds max {}; truncating'.format(size, MAX_STRING_LEN))
        excess = size - MAX_STRING_LEN
        size = MAX_STRING_LEN
    (buf,) = unpack_stream('{}s{}x'.format(size, excess), stream)
    return buf.decode('ascii')

## Classes and data structures

### ClassicRecordFileHeader

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Parcel/ClassicRecordFile.cpp

class ClassicRecordFileHeader:
    def __init__(self, *, title='', sumValid=False, checksum=0, recordsMax=0):
        self.title = title
        self.sumValid = sumValid
        self.checksum = checksum
        self.recordsUsed = 0
        assert recordsMax >= 0
        self.recordsMax = recordsMax # AKA numRecords
        # recordList should be a list of four UInt32
        self.recordList = array('L')
        self.recordList.extend(0 for i in range(recordsMax))

    def __repr__(self):
        return re.sub(r'\s+', ' ', '''
        ClassicRecordFileHeader(
            title={self.title!r},
            sumValid={self.sumValid!r},
            checksum={self.checksum!r},
            recordsMax={self.recordsMax!r},
        )'''.format(self=self)).strip()

    # AKA .Serialize() for writing
    def pack(self) -> bytes:
        raise NotImplementedError('WIP')
        return struct.pack()

    # AKA .Serialize() for reading
    # Modifies in-place
    def unpack_here(self, stream: BinaryIO):
        self.title = unpack_string(stream)
        self.sumValid, self.checksum, self.recordsUsed, self.recordsMax = unpack_stream('<8xLLLL8x', stream)
        self.sumValid = bool(self.sumValid)
        self.recordList = array('L')

        assert self.recordsMax == 4

        # Check title for validity.
        if 'HoverRace track file' not in self.title and 'Fireball object factory resource file' not in self.title:
            raise HoverRaceTrackException('Missing or corrupt header')

        for i in range(self.recordsMax):
            # This is a list of "pointers" to file positions.
            (uint32,) = unpack_stream('<L', stream)
            self.recordList.append(uint32)
            # https://github.com/HoverRace/HoverRace/blob/master/doc/dev/track-format.txt
            # A track has 4 records.
            # Record 0 is a TrackEntry, AKA metadata.
            # Record 1 is a Level.
            # Record 2 is a Background (optional?). [NOT a background palette (8128 * 3 bytes)?]
            # Record 3 is the map (LoadMap), AKA a texture. (optional?)

    # AKA .Serialize() for reading
    # Returns a new instance.
    @classmethod
    def unpack(cls, stream: BinaryIO) -> ClassicRecordFileHeader:
        obj = cls()
        obj.unpack_here(stream)
        return obj

### TrackEntry (metadata)

In [ ]:
MR_REGISTERED_TRACK = 0;
MR_FREE_TRACK = 1;

class RegistrationMode(Enum):
    REGISTERED = 0
    FREE = 1

In [ ]:
MR_NOBITMAP = 0;
MR_RAWBITMAP = 1;

class ImageType(Enum):
    NOBITMAP = 0
    RAWBITMAP = 1

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Model/TrackEntry.cpp

MR_MAGIC_TRACK_NUMBER = 82617

@dataclass
class TrackEntry:
    # name: str = ''  # The name is not part of the file. It's likely coming from the filename.
    description: str = ''
    regMinor: int = 0
    regMajor: int = 0
    registrationMode: RegistrationMode = 0
    sortingIndex: int = 0

    @classmethod
    def unpack(cls, stream: BinaryIO):
        obj = cls()
        magicNumber, version = unpack_stream('<ll', stream)
        if magicNumber != MR_MAGIC_TRACK_NUMBER:
            raise HoverRaceTrackException('Bad magic number: 0x%08x' % magicNumber)
        if version != 1:
            raise HoverRaceTrackException('Unknown track version: {}'.format(magicNumber))
        obj.description = unpack_string(stream)
        obj.regMinor, obj.regMajor, obj.sortingIndex, mode = unpack_stream('<llll', stream)
        obj.registrationMode = RegistrationMode(mode)
        if obj.registrationMode == RegistrationMode.FREE:
            (magicNumber2,) = unpack_stream('<l', stream)
            if magicNumber2 != MR_MAGIC_TRACK_NUMBER:
                raise HoverRaceTrackException('Bad magic number for free track: 0x%08x' % magicNumber2)
        return obj


### Level

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Model/Track.cpp
# https://github.com/HoverRace/HoverRace/blob/master/engine/Model/Level.h
# https://github.com/HoverRace/HoverRace/blob/master/engine/Model/Level.cpp

# TODO: Write this code! It involves in 9 different data structures!
# Classes:
# * Track (maybe already defined elsewhere)
# * Level (the main class here)
# * SectionShape (derived from PolygonShape)
# * FreeElementList
# And also structs:
# * SectionId
# * Section
# * Feature (derived from Section)
# * Room (derived from Section)
# * AudibleRoom

### Color Palette

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/VideoServices/ColorPalette.h
MR_NB_COLORS                 = 256
MR_RESERVED_COLORS_BEGINNING =  10
MR_RESERVED_COLORS_END       =  15
MR_RESERVED_COLORS           =   6
MR_BASIC_COLORS              = 100  # Includes some extra space
MR_BACK_COLORS               = 128

MR_NB_COLOR_INTENSITY        = 256
MR_NORMAL_INTENSITY          = 128

# https://github.com/HoverRace/HoverRace/blob/master/engine/VideoServices/Viewport3D.h
MR_BACK_X_RES = 2048
MR_BACK_Y_RES = 256

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/VideoServices/ColorTab.cpp

BASIC_PALETTE_SIZE = 68
BASIC_PALETTE = [
    [1.000000, 1.000000, 1.000000],
	[0.000000, 0.000000, 0.000000],
	[0.937500, 0.937500, 0.937500],
	[0.875000, 0.875000, 0.875000],
	[0.812500, 0.812500, 0.812500],
	[0.750000, 0.750000, 0.750000],
	[0.687500, 0.687500, 0.687500],
	[0.625000, 0.625000, 0.625000],
	[0.562500, 0.562500, 0.562500],
	[0.500000, 0.500000, 0.500000],
	[0.437500, 0.437500, 0.437500],
	[0.375000, 0.375000, 0.375000],
	[0.312500, 0.312500, 0.312500],
	[0.250000, 0.250000, 0.250000],
	[0.187500, 0.187500, 0.187500],
	[0.125000, 0.125000, 0.125000],
	[0.062500, 0.062500, 0.062500],
	[0.972549, 0.647059, 0.384314],
	[0.875294, 0.582353, 0.345882],
	[0.680784, 0.452941, 0.269020],
	[0.979412, 0.735294, 0.538235],
	[0.349020, 0.803922, 0.270588],
	[0.261765, 0.602941, 0.202941],
	[0.174510, 0.401961, 0.135294],
	[0.674510, 0.901961, 0.635294],
	[0.992157, 0.039216, 0.039216],
	[0.744118, 0.029412, 0.029412],
	[0.496078, 0.019608, 0.019608],
	[0.996078, 0.519608, 0.519608],
	[0.988235, 0.843137, 0.003922],
	[0.741176, 0.632353, 0.002941],
	[0.494118, 0.421569, 0.001961],
	[0.247059, 0.210784, 0.000980],
	[0.890196, 0.023529, 0.243137],
	[0.801176, 0.021176, 0.218824],
	[0.712157, 0.018824, 0.194510],
	[0.623137, 0.016471, 0.170196],
	[0.917647, 0.267647, 0.432353],
	[0.945098, 0.511765, 0.621569],
	[0.972549, 0.755882, 0.810784],
	[0.000000, 0.172549, 0.756863],
	[0.000000, 0.155294, 0.681176],
	[0.000000, 0.138039, 0.605490],
	[0.000000, 0.120784, 0.529804],
	[0.250000, 0.379412, 0.817647],
	[0.500000, 0.586275, 0.878431],
	[0.750000, 0.793137, 0.939216],
	[0.556863, 0.274510, 0.650980],
	[0.501176, 0.247059, 0.585882],
	[0.445490, 0.219608, 0.520784],
	[0.389804, 0.192157, 0.455686],
	[0.667647, 0.455882, 0.738235],
	[0.778431, 0.637255, 0.825490],
	[0.889216, 0.818627, 0.912745],
	[0.968627, 0.968627, 0.000000],
	[0.887908, 0.887908, 0.000000],
	[0.807190, 0.807190, 0.000000],
	[0.726471, 0.726471, 0.000000],
	[0.976471, 0.976471, 0.250000],
	[0.984314, 0.984314, 0.500000],
	[0.992157, 0.992157, 0.750000],
	[1.000000, 0.533333, 0.066667],
	[0.916667, 0.488889, 0.061111],
	[0.833333, 0.444444, 0.055556],
	[0.750000, 0.400000, 0.050000],
	[1.000000, 0.650000, 0.300000],
	[1.000000, 0.766667, 0.533333],
	[1.000000, 0.883333, 0.766667],
]

assert len(BASIC_PALETTE) == BASIC_PALETTE_SIZE

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/VideoServices/VideoBuffer.cpp

def createPalette(raw_pal: bytes) -> ImagePalette:
    # Hard-coded color correction, defaults from:
    # https://github.com/HoverRace/HoverRace/blob/master/engine/Util/Config.cpp
    gamma = 1.2
    contrast = 0.95
    brightness = 0.95
    textScale = 0.7

    # TODO: This function is very messy.
    # I should refactor it:
    # * One function to do color correction. Could be a lambda or partial function.
    # * One function to generate the basic palette.
    # * One function to generate the bg palette. (Optionally without any color correction.)
    # * One function to join it all together (concatenation of bytearrays).

    pGamma = 1.0 / gamma
    pIntensity = contrast * brightness
    pIntensityBase = brightness - pIntensity

    pal = bytearray(256 * 3)
    idx = MR_RESERVED_COLORS_BEGINNING * 3
    for component in chain.from_iterable(BASIC_PALETTE):
        corrected = min(255, int(256 * (pIntensityBase + pIntensity * (component ** pGamma))))
        pal[idx] = corrected
        idx += 1
    assert idx % 3 == 0
    while idx - (MR_RESERVED_COLORS_BEGINNING * 3) < MR_BASIC_COLORS * 3:
        x = idx // 3
        pal[idx + 0] = 255  # Red
        pal[idx + 1] = 255  # Green
        pal[idx + 2] = x - 15  # Blue
        idx += 3  # Skipping one, for some reason.
        idx += 3
    assert idx == (MR_RESERVED_COLORS_BEGINNING + MR_BASIC_COLORS) * 3

    assert len(raw_pal) == MR_BACK_COLORS * 3
    for component in raw_pal:
        corrected = min(255, int(256 * (pIntensityBase + pIntensity * ((component / 256) ** pGamma))))
        pal[idx] = corrected
        idx += 1

    return ImagePalette('RGB', pal)

### Background

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/client/Game2/ClientSession.cpp

@dataclass
class Background:
    imageType: ImageType = 0
    raw_pal: bytes = field(default=None, repr=False)
    raw_img: bytes = field(default=None, repr=False)
    pal: ImagePalette = None
    img: Image = None

    @classmethod
    def unpack(cls, stream:BinaryIO):
        obj = cls()
        (img_type,) = unpack_stream('<l', stream)
        obj.imageType = ImageType(img_type)
        if obj.imageType == ImageType.RAWBITMAP:
            obj.raw_pal, obj.raw_img = unpack_stream('{}s{}s'.format(MR_BACK_COLORS * 3, MR_BACK_X_RES * MR_BACK_Y_RES), stream)
            # The background image doesn't use any colors outside the correct range.
            assert all(
                MR_RESERVED_COLORS_BEGINNING + MR_BASIC_COLORS <= color < MR_RESERVED_COLORS_BEGINNING + MR_BASIC_COLORS + MR_BACK_COLORS
                for color in obj.raw_img
            )
            obj.pal = createPalette(obj.raw_pal)
            obj.img = Image_frombytes('P', (MR_BACK_Y_RES, MR_BACK_X_RES), obj.raw_img).transpose(Transpose.ROTATE_90)
            obj.img.putpalette(obj.pal)
        return obj

### Track Map

#### SpriteTextureRes

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Display/SpriteTextureRes.cpp

MAX_TEXTURE_WIDTH = 4096
MAX_TEXTURE_HEIGHT = 4096

@dataclass
class SpriteTextureRes:
    numItems: int = 0
    itemHeight: int = 0
    totalHeight: int = 0
    width: int = 0
    img: Image = None

    @classmethod
    def unpack(cls, stream:BinaryIO):
        obj = cls()
        obj.numItems, obj.itemHeight, obj.totalHeight, obj.width = unpack_stream('<LLLL', stream)

        if obj.numItems == 0:
            raise HoverRaceTrackException('No items in sprite')
        elif obj.numItems > 1:
            warn('More than one item in sprite texture (entire sprite will be used): {}'.format(obj.numItems))

        if obj.width > MAX_TEXTURE_WIDTH or obj.totalHeight > MAX_TEXTURE_HEIGHT:
            raise HoverRaceTrackException('Texture size ({}x{}) exceeds maximum size ({}x{})'.format(obj.width, obj.totalHeight, MAX_TEXTURE_WIDTH, MAX_TEXTURE_HEIGHT))

        assert obj.itemHeight <= obj.totalHeight
        assert obj.totalHeight % obj.itemHeight == 0

        (raw_img,) = unpack_stream('{}s'.format(obj.width * obj.totalHeight), stream)
        # Should I use the basic palette for this one?
        obj.img = Image_frombytes('L', (obj.width, obj.totalHeight), raw_img)
        return obj

#### Track

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Model/Track.cpp
# https://github.com/HoverRace/HoverRace/blob/master/engine/Parcel/TrackBundle.cpp

@dataclass
class Track:
    x0: int = 0
    x1: int = 0
    y0: int = 0
    y1: int = 0
    map: SpriteTextureRes = None

    @property
    def width(self):
        return self.x1 - self.x0
    @property
    def height(self):
        return self.y1 - self.y0

    @classmethod
    def unpack(cls, stream:BinaryIO):
        obj = cls()
        obj.x0, obj.x1, obj.y0, obj.y1 = unpack_stream('<llll', stream)
        obj.map = SpriteTextureRes.unpack(stream)
        return obj

### ClassicRecordFile (TRK file)

In [ ]:
# https://github.com/HoverRace/HoverRace/blob/master/engine/Parcel/ClassicRecordFile.cpp

@dataclass
class ClassicRecordFile:
    header: ClassicRecordFileHeader = None
    entry: TrackEntry = None
    level = None
    background: Background = None
    track: Track = None

    @classmethod
    def unpack(cls, stream: BinaryIO):
        obj = cls()
        obj.header = ClassicRecordFileHeader.unpack(stream)
        stream.seek(obj.header.recordList[0])
        obj.entry = TrackEntry.unpack(stream)
        stream.seek(obj.header.recordList[1])
        # TODO
        stream.seek(obj.header.recordList[2])
        obj.background = Background.unpack(stream)
        stream.seek(obj.header.recordList[3])
        obj.track = Track.unpack(stream)
        return obj

## Main application-like code

In [ ]:
BASEDIR = Path('~/.var/app/com.valvesoftware.Steam/data/Steam/steamapps/common/HoverRace/tracks/').expanduser()

In [ ]:
with open(BASEDIR / 'ClassicH.trk', 'rb') as f:
    record = ClassicRecordFile.unpack(f)

In [ ]:
record

In [ ]:
[hex(x) for x in record.header.recordList]

In [ ]:
record.background.raw_pal

In [ ]:
record.background.pal.getdata()

In [ ]:
record.background.img

In [ ]:
record.track.map.img